> Simulations proceed as follows. At each round, a random subset of users (20 000, in the following) is presented to
several sequential algorithms a.k.a. policies to be evaluated. These policies must then recommend an ordered set of
L = 12 playlists to each user. Streams, i.e. positive binary rewards, are generated according to the aforementioned
display-to-stream probabilities and to a configurable cascading browsing model capturing that users explore the carousel
from left to right and might not see all recommended playlists. At the end of each round, all policies update their model
based on the set of users and on binary rewards received from displayed playlists. Expected cumulative regrets of
policies [2, 22, 39] w.r.t. the optimal top-L playlists sets according to pui probabilities are computed.

> An active user who did not stream any card during the round only saw the Linit first ones.
• An active user who streamed the ith card, with i ∈ {1, ..., L}, saw all cards from ranks 1 to max(Linit,i).

[[Source]](https://arxiv.org/pdf/2009.06546.pdf)

In [1]:
from sd_bandits.deezer.dataset import DeezerDataset
import numpy as np

In [2]:
deezer_data = DeezerDataset(
    user_features="../data/deezer_toy/user_features_small.csv",
    playlist_features= "../data/deezer_toy/playlist_features.csv"
)

In [3]:
feedback = deezer_data.obtain_batch_bandit_feedback()

100%|██████████| 20000/20000 [00:00<00:00, 63017.70it/s]


In [4]:
print("feedback dict:")
for key, value in feedback.items():
    print(f"  {key}: {type(value)}")

feedback dict:
  action: <class 'numpy.ndarray'>
  reward: <class 'numpy.ndarray'>
  position: <class 'numpy.ndarray'>
  context: <class 'numpy.ndarray'>
  action_context: <class 'numpy.ndarray'>
  pscore: <class 'numpy.ndarray'>
